In [772]:
import urllib
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
from collections import defaultdict

In [773]:
URL = 'https://haraj.com.sa/tags/%D8%AD%D8%B1%D8%A7%D8%AC%20%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B1%D8%A7%D8%AA'
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib') 

table = soup.findAll('a',attrs = {'class':'brand-logos--svg'})


In [774]:
# table

In [775]:
df = pd.DataFrame(columns= ['Company', 'Carmodel', 'URL'])

company_names = []
car_names = []

# print(table[0]['href'])

company_links = defaultdict(list)
for link in table:
    href = link['href']
    URL = 'https://haraj.com.sa' + urllib.parse.quote(href)
    company_name = href.split('/')[-1]

    company_links[company_name].append(URL)


In [776]:
company_df = pd.DataFrame(company_links.items(), columns=['company','url'])
company_df.head()

,company,url
0,تويوتا,[https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%...
1,نيسان,[https://haraj.com.sa/tags/%D9%86%D9%8A%D8%B3%...
2,فورد,[https://haraj.com.sa/tags/%D9%81%D9%88%D8%B1%...
3,لكزس,[https://haraj.com.sa/tags/%D9%84%D9%83%D8%B2%...
4,شيفروليه,[https://haraj.com.sa/tags/%D8%B4%D9%8A%D9%81%...


In [786]:
df = pd.DataFrame(columns=['company','company_url','car_name','car_link'])




for company, href in company_links.items():

    URL = href[0] #'https://haraj.com.sa' + urllib.parse.quote(href)
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html5lib') 

    cars2 = soup.findAll('ul', attrs = {'class':'tabs_list supTabs list-container'})
    
    try:
        for i in cars2[1]:
            car_type = i.findAll("a")#[0].text 
            car_type = [c.text for c in car_type]

            car_link = i.find_all('a',attrs={'class': 'supTab_link'})[0]['href']
            
            company_links[company].append(car_type)
            company_links[company].append(car_link)
            
            df = df.append({'company':company ,
                'company_url': URL,
                'car_name': car_type[0],
                'car_link':car_link }
               , ignore_index=True)
    except:
        car_type = 'Unknown'
        car_link = 'Unknown'
        df = df.append({'company':company ,
                            'company_url': URL,
                            'car_name': car_type[0],
                            'car_link':car_link }
                           , ignore_index=True)
        
        company_links[company].append(car_type)
        company_links[company].append(car_link)
    
    

In [793]:
# company_links
df = df.drop_duplicates()
print(df.shape)
print(df.columns)

(591, 4)
Index(['company', 'company_url', 'car_name', 'car_link'], dtype='object')


In [791]:
un_wanted_links = ['قطع غيار وملحقات',
'شاحنات ومعدات ثقيلة',
'دبابات',
'سيارات تراثية',
'search-مصدوم مصدومة',
'search-للتنازل تنازل',
'0',
'1',
0,
1]


In [796]:
df = df[~df['company'].isin(un_wanted_links)]
print(df.shape)
print(df.columns)

(557, 4)
Index(['company', 'company_url', 'car_name', 'car_link'], dtype='object')


In [797]:
df

,company,company_url,car_name,car_link
0,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,C-HR,/tags/C-HR
1,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,جرانفيا,/tags/جرانفيا
2,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,راش,/tags/راش
3,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,بريوس,/tags/بريوس
4,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,هايلاندر,/tags/هايلاندر
...,...,...,...,...
552,فولفو,https://haraj.com.sa/tags/%D9%81%D9%88%D9%84%D...,XC90,/tags/XC90
553,فولفو,https://haraj.com.sa/tags/%D9%81%D9%88%D9%84%D...,850,/tags/850
554,فولفو,https://haraj.com.sa/tags/%D9%81%D9%88%D9%84%D...,S 80,/tags/S 80
555,ZXAUTO,https://haraj.com.sa/tags/ZXAUTO,جرامد تايجر,/tags/جرامد تايجر


In [1001]:
def get_posts(car_link):
    list_cars = []
    car_link = str(car_link)
    
    if car_link == 'Unknown':
        return 'Unknown'
    
    for page_number in range(1,6):
        link = 'https://legacy.haraj.com.sa'+car_link+'/'+ str(page_number)+'/'
        r = requests.get(link) 
        soup = BeautifulSoup(r.content, 'html5lib') 
        soup = soup.findAll('div', attrs = {'class':'adxTitle'})

#         return soup
        
        try:
            for i in soup:
                list_cars.append(i.findAll('a')[0]['href'])
        except:
            list_cars.append('no_links')
        break
    return list_cars
# soup = get_posts('/tags/C-HR')
# adxInfo

In [1002]:
# df[df['car_link'] == '/tags/C-HR']['car_link'].apply(get_posts)
# df_test = df.head(2)

In [1003]:
df['cars_posts'] = df['car_link'].apply(get_posts)

In [1004]:
df.head()

,company,company_url,car_name,car_link,cars_posts
0,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,C-HR,/tags/C-HR,[https://legacy.haraj.com.sa/1172470065/حجر_عي...
1,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,جرانفيا,/tags/جرانفيا,[https://legacy.haraj.com.sa/1172125750/تويوتا...
2,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,راش,/tags/راش,[https://legacy.haraj.com.sa/1172455829/تويوتا...
3,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,بريوس,/tags/بريوس,[https://legacy.haraj.com.sa/1172393338/تويوتا...
4,تويوتا,https://haraj.com.sa/tags/%D8%AA%D9%88%D9%8A%D...,هايلاندر,/tags/هايلاندر,[https://legacy.haraj.com.sa/1172458691/هايلان...


In [1005]:
def get_images(posts):
    img_list = []
    for link in posts:

        r = requests.get(link) 
        soup = BeautifulSoup(r.content, 'html5lib') 
        soup = soup.findAll('div', attrs = {'class':'adxBody'})
        
        for i in soup:
            img = i.findAll('img')
            for im in img:
                img_list.append(im['src'])
                
    return img_list

In [ ]:
df['img_link'] = df['cars_posts'].apply(get_images)

In [ ]:
df.head()

In [ ]:
df = df.explode('img_link')
df = df.reset_index()